## Import Libraries

In [1]:
from utils.data import Data
import numpy as np
import matplotlib.pyplot as plt
import rasterio
np.random.seed(7)

## Define functions

In [2]:
def get_acc(true, pred, channel):
    pred[true == 0] = 0
    correct = np.sum((true == channel)*(pred==channel))
    total = np.sum(true==channel)
    return correct/total

def get_precision(true, pred, channel):
    pred[true == 0] = 0
    tp = np.sum((true == channel)*(pred==channel))
    fp = np.sum((true != channel)*(pred==channel))
    return tp/(tp+fp)

def get_recall(true, pred, channel):
    pred[true == 0] = 0
    tp = np.sum((true == channel)*(pred==channel))
    fn = np.sum((true == channel)*(pred!=channel))
    return tp/(tp+fn)

def get_fscore(true, pred, channel):
    pred[true == 0] = 0
    precision = get_precision(true, pred, channel)
    recall = get_recall(true, pred, channel)
    return 2*precision*recall/(precision+recall)

def get_acc_precision_recall_fscore(true, pred, channel):
    acc = get_acc(true, pred, channel)
    precision = get_precision(true, pred, channel)
    recall = get_recall(true, pred, channel)
    fscore = get_fscore(true, pred, channel)
    return round(acc, 4), round(precision, 4), round(recall, 4), round(fscore, 4)

classes = sorted(["Vegetation", "Bare Ground", "Shade"])

## Jornada Site C Post harvest

In [3]:
c_post_tiff_filename = "./Sergio/Images/JOR_C_Post.tif"
c_post_shp_filename = "./Sergio/Labels/JOR_C_Post.shp"
c_post_data = Data(c_post_tiff_filename, c_post_shp_filename, classes = classes)
c_post_tiff = c_post_data.read_tiff()
c_post_mask = c_post_data.get_mask(column="Classname", drop="Other")
nil = np.sum(c_post_mask, axis=2) == 0
ground_truth = np.argmax(c_post_mask, axis=2)+1
ground_truth[nil] = 0
# Ground Truth classes
#   0 : Background
#   1 : Bare ground
#   2 : Shade
#   3 : Vegitation
max_likelihood_file = "./Sergio/max_likelihood/JOR_C_Post.tif"
max_likelihood_tiff = rasterio.open(max_likelihood_file)
max_likelihood_np = max_likelihood_tiff.read()
max_likelihood_np = np.squeeze(max_likelihood_np)
# Max likelihood classes
#   0 : Background
#   1 : Bare ground
#   2 : Shade
#   3 : Vegitation
max_likelihood_np[max_likelihood_np==1] = 30
max_likelihood_np[max_likelihood_np==2] = 1
max_likelihood_np[max_likelihood_np==3] = 2
max_likelihood_np[max_likelihood_np==30] = 3
random_forest_file = "./outputs/c_post_all.tif"
random_forest_tiff = rasterio.open(random_forest_file)
random_forest_np = random_forest_tiff.read()
random_forest_np = np.squeeze(random_forest_np)
from collections import Counter
Counter(random_forest_np.flatten())
print("Maximum Likelihood")
for i,label in enumerate(classes):
    print(f"\t{label}\t| Accuracy, Precision, Recall, FScore:{get_acc_precision_recall_fscore(ground_truth, max_likelihood_np, i+1)}")
print("Random Forest")
for i,label in enumerate(classes):
    print(f"\t{label}\t| Accuracy, Precision, Recall, FScore:{get_acc_precision_recall_fscore(ground_truth, random_forest_np, i+1)}")

/home/mibook/env/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Maximum Likelihood
	Bare Ground	| Accuracy, Precision, Recall, FScore:(0.9914, 0.9685, 0.9914, 0.9798)
	Shade	| Accuracy, Precision, Recall, FScore:(0.9335, 0.9065, 0.9335, 0.9198)
	Vegetation	| Accuracy, Precision, Recall, FScore:(0.949, 0.9744, 0.949, 0.9615)
Random Forest
	Bare Ground	| Accuracy, Precision, Recall, FScore:(0.9907, 0.9673, 0.9907, 0.9789)
	Shade	| Accuracy, Precision, Recall, FScore:(0.9654, 0.8773, 0.9654, 0.9192)
	Vegetation	| Accuracy, Precision, Recall, FScore:(0.9328, 0.9873, 0.9328, 0.9592)


## Jornada Site M Pre Harvest

In [4]:
m_pre_tiff_filename = "./Sergio/Images/JOR_M_Pre.tif"
m_pre_shp_filename = "./Sergio/Labels/JOR_M_Pre.shp"
m_pre_data = Data(m_pre_tiff_filename, m_pre_shp_filename, classes = classes)
m_pre_tiff = m_pre_data.read_tiff()
m_pre_mask = m_pre_data.get_mask(column="Classname", drop="Other")
nil = np.sum(m_pre_mask, axis=2) == 0
ground_truth = np.argmax(m_pre_mask, axis=2)+1
ground_truth[nil] = 0
# Ground Truth classes
#   0 : Bare ground
#   1 : Shade
#   2 : Vegitation
max_likelihood_file = "./Sergio/max_likelihood/JOR_M_Pre.tif"
max_likelihood_tiff = rasterio.open(max_likelihood_file)
max_likelihood_np = max_likelihood_tiff.read()
max_likelihood_np = np.squeeze(max_likelihood_np)
# Max likelihood classes
#   0 : Background
#   1 : Bare ground
#   2 : Shade
#   3 : Vegitation
max_likelihood_np[max_likelihood_np==1] = 30
max_likelihood_np[max_likelihood_np==2] = 1
max_likelihood_np[max_likelihood_np==3] = 2
max_likelihood_np[max_likelihood_np==30] = 3
random_forest_file = "./outputs/m_pre_all.tif"
random_forest_tiff = rasterio.open(random_forest_file)
random_forest_np = random_forest_tiff.read()
random_forest_np = np.squeeze(random_forest_np)
from collections import Counter
Counter(random_forest_np.flatten())
print("Maximum Likelihood")
ground_truth = ground_truth[:-1,:-1]
random_forest_np = random_forest_np[:-1,:-1]
for i,label in enumerate(classes):
    print(f"\t{label}\t| Accuracy, Precision, Recall, FScore:{get_acc_precision_recall_fscore(ground_truth, max_likelihood_np, i+1)}")
print("Random Forest")
for i,label in enumerate(classes):
    print(f"\t{label}\t| Accuracy, Precision, Recall, FScore:{get_acc_precision_recall_fscore(ground_truth, random_forest_np, i+1)}")

Maximum Likelihood
	Bare Ground	| Accuracy, Precision, Recall, FScore:(0.9932, 0.993, 0.9932, 0.9931)
	Shade	| Accuracy, Precision, Recall, FScore:(0.9671, 0.9602, 0.9671, 0.9636)
	Vegetation	| Accuracy, Precision, Recall, FScore:(0.987, 0.9902, 0.987, 0.9886)
Random Forest
	Bare Ground	| Accuracy, Precision, Recall, FScore:(0.8721, 0.7654, 0.8721, 0.8153)
	Shade	| Accuracy, Precision, Recall, FScore:(0.4721, 0.5545, 0.4721, 0.51)
	Vegetation	| Accuracy, Precision, Recall, FScore:(0.9076, 0.9532, 0.9076, 0.9299)


## Jornada Site M Post harvest

In [5]:
m_post_tiff_filename = "./Sergio/Images/JOR_M_Post.tif"
m_post_shp_filename = "./Sergio/Labels/JOR_M_Post.shp"
m_post_data = Data(m_post_tiff_filename, m_post_shp_filename, classes = classes)
m_post_tiff = m_post_data.read_tiff()
m_post_mask = m_post_data.get_mask(column="Classname", drop="Other")
nil = np.sum(m_post_mask, axis=2) == 0
ground_truth = np.argmax(m_post_mask, axis=2)+1
ground_truth[nil] = 0
# Ground Truth classes
#   0 : Background
#   1 : Bare ground
#   2 : Shade
#   3 : Vegitation
max_likelihood_file = "./Sergio/max_likelihood/JOR_M_Post.tif"
max_likelihood_tiff = rasterio.open(max_likelihood_file)
max_likelihood_np = max_likelihood_tiff.read()
max_likelihood_np = np.squeeze(max_likelihood_np)
# Max likelihood classes
#   0 : Background
#   1 : Bare ground
#   2 : Shade
#   3 : Vegitation
max_likelihood_np[max_likelihood_np==1] = 30
max_likelihood_np[max_likelihood_np==2] = 1
max_likelihood_np[max_likelihood_np==3] = 2
max_likelihood_np[max_likelihood_np==30] = 3
random_forest_file = "./outputs/m_post_all.tif"
random_forest_tiff = rasterio.open(random_forest_file)
random_forest_np = random_forest_tiff.read()
random_forest_np = np.squeeze(random_forest_np)
from collections import Counter
Counter(random_forest_np.flatten())
print("Maximum Likelihood")
for i,label in enumerate(classes):
    print(f"\t{label}\t| Accuracy, Precision, Recall, FScore:{get_acc_precision_recall_fscore(ground_truth, max_likelihood_np, i+1)}")
print("Random Forest")
for i,label in enumerate(classes):
    print(f"\t{label}\t| Accuracy, Precision, Recall, FScore:{get_acc_precision_recall_fscore(ground_truth, random_forest_np, i+1)}")

Maximum Likelihood
	Bare Ground	| Accuracy, Precision, Recall, FScore:(0.9963, 0.9955, 0.9963, 0.9959)
	Shade	| Accuracy, Precision, Recall, FScore:(0.9816, 0.9908, 0.9816, 0.9862)
	Vegetation	| Accuracy, Precision, Recall, FScore:(0.9962, 0.9942, 0.9962, 0.9952)
Random Forest
	Bare Ground	| Accuracy, Precision, Recall, FScore:(0.9974, 0.9968, 0.9974, 0.9971)
	Shade	| Accuracy, Precision, Recall, FScore:(0.996, 0.9783, 0.996, 0.9871)
	Vegetation	| Accuracy, Precision, Recall, FScore:(0.9941, 0.9991, 0.9941, 0.9966)


## Jornada Site P Pre Harvest

In [6]:
p_pre_tiff_filename = "./Sergio/Images/JOR_P_Pre.tif"
p_pre_shp_filename = "./Sergio/Labels/JOR_P_Pre.shp"
p_pre_data = Data(p_pre_tiff_filename, p_pre_shp_filename, classes = classes)
p_pre_tiff = p_pre_data.read_tiff()
p_pre_mask = p_pre_data.get_mask(column="Classname", drop="Other")
nil = np.sum(p_pre_mask, axis=2) == 0
ground_truth = np.argmax(p_pre_mask, axis=2)+1
ground_truth[nil] = 0
# Ground Truth classes
#   0 : Background
#   1 : Bare ground
#   2 : Shade
#   3 : Vegitation
max_likelihood_file = "./Sergio/max_likelihood/JOR_P_Pre.tif"
max_likelihood_tiff = rasterio.open(max_likelihood_file)
max_likelihood_np = max_likelihood_tiff.read()
max_likelihood_np = np.squeeze(max_likelihood_np)
# Max likelihood classes
#   0 : Background
#   1 : Bare ground
#   2 : Shade
#   3 : Vegitation
max_likelihood_np[max_likelihood_np==1] = 30
max_likelihood_np[max_likelihood_np==2] = 1
max_likelihood_np[max_likelihood_np==3] = 2
max_likelihood_np[max_likelihood_np==30] = 3
random_forest_file = "./outputs/p_pre_all.tif"
random_forest_tiff = rasterio.open(random_forest_file)
random_forest_np = random_forest_tiff.read()
random_forest_np = np.squeeze(random_forest_np)
from collections import Counter
Counter(random_forest_np.flatten())
print("Maximum Likelihood")
ground_truth = ground_truth[:-1,:-1]
random_forest_np = random_forest_np[:-1,:-1]
for i,label in enumerate(classes):
    print(f"\t{label}\t| Accuracy, Precision, Recall, FScore:{get_acc_precision_recall_fscore(ground_truth, max_likelihood_np, i+1)}")
print("Random Forest")
for i,label in enumerate(classes):
    print(f"\t{label}\t| Accuracy, Precision, Recall, FScore:{get_acc_precision_recall_fscore(ground_truth, random_forest_np, i+1)}")

Maximum Likelihood
	Bare Ground	| Accuracy, Precision, Recall, FScore:(0.9978, 0.9952, 0.9978, 0.9965)
	Shade	| Accuracy, Precision, Recall, FScore:(0.929, 0.9636, 0.929, 0.946)
	Vegetation	| Accuracy, Precision, Recall, FScore:(0.9825, 0.9731, 0.9825, 0.9778)
Random Forest
	Bare Ground	| Accuracy, Precision, Recall, FScore:(0.9946, 0.9987, 0.9946, 0.9967)
	Shade	| Accuracy, Precision, Recall, FScore:(0.9531, 0.8881, 0.9531, 0.9195)
	Vegetation	| Accuracy, Precision, Recall, FScore:(0.9621, 0.983, 0.9621, 0.9724)


## Jornada Site P Post Harvest

In [7]:
p_post_tiff_filename = "./Sergio/Images/JOR_P_Post.tif"
p_post_shp_filename = "./Sergio/Labels/JOR_P_Post.shp"
p_post_data = Data(p_post_tiff_filename, p_post_shp_filename, classes = classes)
p_post_tiff = p_post_data.read_tiff()
p_post_mask = p_post_data.get_mask(column="Classname", drop="Other")
nil = np.sum(p_post_mask, axis=2) == 0
ground_truth = np.argmax(p_post_mask, axis=2)+1
ground_truth[nil] = 0
# Ground Truth classes
#   0 : Background
#   1 : Bare ground
#   2 : Shade
#   3 : Vegitation
max_likelihood_file = "./Sergio/max_likelihood/JOR_P_Post.tif"
max_likelihood_tiff = rasterio.open(max_likelihood_file)
max_likelihood_np = max_likelihood_tiff.read()
max_likelihood_np = np.squeeze(max_likelihood_np)
# Max likelihood classes
#   0 : Background
#   1 : Bare ground
#   2 : Shade
#   3 : Vegitation
max_likelihood_np[max_likelihood_np==1] = 30
max_likelihood_np[max_likelihood_np==2] = 1
max_likelihood_np[max_likelihood_np==3] = 2
max_likelihood_np[max_likelihood_np==30] = 3
random_forest_file = "./outputs/p_post_all.tif"
random_forest_tiff = rasterio.open(random_forest_file)
random_forest_np = random_forest_tiff.read()
random_forest_np = np.squeeze(random_forest_np)
from collections import Counter
print("Maximum Likelihood")
for i,label in enumerate(classes):
    print(f"\t{label}\t| Accuracy, Precision, Recall, FScore:{get_acc_precision_recall_fscore(ground_truth, max_likelihood_np, i+1)}")
print("Random Forest")
for i,label in enumerate(classes):
    print(f"\t{label}\t| Accuracy, Precision, Recall, FScore:{get_acc_precision_recall_fscore(ground_truth, random_forest_np, i+1)}")

Maximum Likelihood
	Bare Ground	| Accuracy, Precision, Recall, FScore:(0.994, 0.9967, 0.994, 0.9954)
	Shade	| Accuracy, Precision, Recall, FScore:(0.9572, 0.9871, 0.9572, 0.9719)
	Vegetation	| Accuracy, Precision, Recall, FScore:(0.9925, 0.9756, 0.9925, 0.984)
Random Forest
	Bare Ground	| Accuracy, Precision, Recall, FScore:(0.9916, 0.9976, 0.9916, 0.9946)
	Shade	| Accuracy, Precision, Recall, FScore:(0.9819, 0.9618, 0.9819, 0.9717)
	Vegetation	| Accuracy, Precision, Recall, FScore:(0.9881, 0.9879, 0.9881, 0.988)


## Jornada Site T Pre Harvest

In [8]:
t_pre_tiff_filename = "./Sergio/Images/JOR_T_Pre.tif"
t_pre_shp_filename = "./Sergio/Labels/JOR_T_Pre.shp"
t_pre_data = Data(t_pre_tiff_filename, t_pre_shp_filename, classes = classes)
t_pre_tiff = t_pre_data.read_tiff()
t_pre_mask = t_pre_data.get_mask(column="Classname", drop="Other")
nil = np.sum(t_pre_mask, axis=2) == 0
ground_truth = np.argmax(t_pre_mask, axis=2)+1
ground_truth[nil] = 0
# Ground Truth classes
#   0 : Background
#   1 : Bare ground
#   2 : Shade
#   3 : Vegitation
max_likelihood_file = "./Sergio/max_likelihood/JOR_T_Pre.tif"
max_likelihood_tiff = rasterio.open(max_likelihood_file)
max_likelihood_np = max_likelihood_tiff.read()
max_likelihood_np = np.squeeze(max_likelihood_np)
# Max likelihood classes
#   0 : Background
#   1 : Bare ground
#   2 : Shade
#   3 : Vegitation
max_likelihood_np[max_likelihood_np==1] = 30
max_likelihood_np[max_likelihood_np==2] = 1
max_likelihood_np[max_likelihood_np==3] = 2
max_likelihood_np[max_likelihood_np==30] = 3
random_forest_file = "./outputs/t_pre_all.tif"
random_forest_tiff = rasterio.open(random_forest_file)
random_forest_np = random_forest_tiff.read()
random_forest_np = np.squeeze(random_forest_np)
from collections import Counter
print("Maximum Likelihood")
ground_truth = ground_truth[:-1,:]
random_forest_np = random_forest_np[:-1,:]
for i,label in enumerate(classes):
    print(f"\t{label}\t| Accuracy, Precision, Recall, FScore:{get_acc_precision_recall_fscore(ground_truth, max_likelihood_np, i+1)}")
print("Random Forest")
for i,label in enumerate(classes):
    print(f"\t{label}\t| Accuracy, Precision, Recall, FScore:{get_acc_precision_recall_fscore(ground_truth, random_forest_np, i+1)}")

Maximum Likelihood
	Bare Ground	| Accuracy, Precision, Recall, FScore:(0.9969, 0.9888, 0.9969, 0.9929)
	Shade	| Accuracy, Precision, Recall, FScore:(0.8504, 0.7347, 0.8504, 0.7883)
	Vegetation	| Accuracy, Precision, Recall, FScore:(0.867, 0.9374, 0.867, 0.9008)
Random Forest
	Bare Ground	| Accuracy, Precision, Recall, FScore:(0.9928, 0.9915, 0.9928, 0.9922)
	Shade	| Accuracy, Precision, Recall, FScore:(0.8919, 0.6398, 0.8919, 0.7451)
	Vegetation	| Accuracy, Precision, Recall, FScore:(0.7986, 0.9396, 0.7986, 0.8634)


In [9]:
t_post_tiff_filename = "./Sergio/Images/JOR_T_Post.tif"
t_post_shp_filename = "./Sergio/Labels/JOR_T_Post.shp"
t_post_data = Data(t_post_tiff_filename, t_post_shp_filename, classes = classes)
t_post_tiff = t_post_data.read_tiff()
t_post_mask = t_post_data.get_mask(column="Classname", drop="Other")
nil = np.sum(t_post_mask, axis=2) == 0
ground_truth = np.argmax(t_post_mask, axis=2)+1
ground_truth[nil] = 0
# Ground Truth classes
#   0 : Background
#   1 : Bare ground
#   2 : Shade
#   3 : Vegitation
max_likelihood_file = "./Sergio/max_likelihood/JOR_T_Post.tif"
max_likelihood_tiff = rasterio.open(max_likelihood_file)
max_likelihood_np = max_likelihood_tiff.read()
max_likelihood_np = np.squeeze(max_likelihood_np)
# Max likelihood classes
#   0 : Background
#   1 : Bare ground
#   2 : Shade
#   3 : Vegitation
max_likelihood_np[max_likelihood_np==1] = 30
max_likelihood_np[max_likelihood_np==2] = 1
max_likelihood_np[max_likelihood_np==3] = 2
max_likelihood_np[max_likelihood_np==30] = 3
random_forest_file = "./outputs/t_post_all.tif"
random_forest_tiff = rasterio.open(random_forest_file)
random_forest_np = random_forest_tiff.read()
random_forest_np = np.squeeze(random_forest_np)
from collections import Counter
print("Maximum Likelihood")
for i,label in enumerate(classes):
    print(f"\t{label}\t| Accuracy, Precision, Recall, FScore:{get_acc_precision_recall_fscore(ground_truth, max_likelihood_np, i+1)}")
print("Random Forest")
for i,label in enumerate(classes):
    print(f"\t{label}\t| Accuracy, Precision, Recall, FScore:{get_acc_precision_recall_fscore(ground_truth, random_forest_np, i+1)}")

Maximum Likelihood
	Bare Ground	| Accuracy, Precision, Recall, FScore:(0.9972, 0.9958, 0.9972, 0.9965)
	Shade	| Accuracy, Precision, Recall, FScore:(0.9294, 0.8859, 0.9294, 0.9071)
	Vegetation	| Accuracy, Precision, Recall, FScore:(0.9541, 0.9739, 0.9541, 0.9639)
Random Forest
	Bare Ground	| Accuracy, Precision, Recall, FScore:(0.9947, 0.9957, 0.9947, 0.9952)
	Shade	| Accuracy, Precision, Recall, FScore:(0.9629, 0.7634, 0.9629, 0.8516)
	Vegetation	| Accuracy, Precision, Recall, FScore:(0.892, 0.9817, 0.892, 0.9347)
